In [ ]:
# делаем import необходимых библиотек
import os

import mlflow

# устанавливаем локальное хранилище для наших экспериментов
# хранилище должно быть такое же, как и при запуске сервиса
mlflow.set_tracking_uri('file:./mlflow_experiments_store')

# получаем id эксеримента, который создаётся по умолчанию
# эксперимент по умолчанию называется Default
experiment_id = mlflow.get_experiment_by_name("Default").experiment_id

# залогируем тестовую метрику и артефакт
with mlflow.start_run(run_name='Default', experiment_id=experiment_id) as run:
    run_id = run.info.run_id
    mlflow.log_metric("test_metric", 0)
    mlflow.log_artifact("test_artifact.txt", "test_artifact")

print(f"Run id запуска: {run_id}")

In [ ]:
!ls mlflow_experiments_store/0/{run_id}/metrics

In [17]:
import os

import mlflow
import pandas as pd
import joblib


EXPERIMENT_NAME = 'bogomolov'
RUN_NAME = "model_0_registry"
REGISTRY_MODEL_NAME = "churn_model_nikolaibogomolov"


os.environ["MLFLOW_S3_ENDPOINT_URL"] = 'https://storage.yandexcloud.net'
os.environ["AWS_ACCESS_KEY_ID"] = 'YCAJE8CKXGh8XKUmcmGHpV2CX'
os.environ["AWS_SECRET_ACCESS_KEY"] = 'YCNQmCkW71PeIVypo2YsF-4eIHhiRBZZN94pztrj'

data = pd.read_csv('initial_data.csv')
data.drop(columns = ['building_id', 'studio', 'building_type_int', 'latitude', 'longitude'], inplace = True)
data_test = data[-100:]
X_test = data_test.drop(columns = ['price'])
y_test = data_test['price']

with open('fitted_model.pkl', 'rb') as fd:
    model = joblib.load(fd) 

prediction = y_test
probas = y_test

pip_requirements = 'requirements.txt'
signature = mlflow.models.infer_signature(X_test, prediction)
input_example = X_test[:10]
metadata = {'model_type': 'monthly'}


experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id
if not experiment_id:
    experiment_id = mlflow.create_experiment(EXPERIMENT_NAME).experiment_id

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    run_id = run.info.run_id

    model_info = mlflow.sklearn.log_model( 
        sk_model=model,
        artifact_path="models",
        registered_model_name=REGISTRY_MODEL_NAME,
        signature = signature,
        pip_requirements=pip_requirements,
        input_example = input_example,
        metadata = metadata,
        await_registration_for=60       
    )
    
    

/home/mle-user/mle_projects/mle-mlflow/.venv_flow/lib/python3.10/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator OneHotEncoder from version 1.4.0 when using version 1.3.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/mle-user/mle_projects/mle-mlflow/.venv_flow/lib/python3.10/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.4.0 when using version 1.3.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/mle-user/mle_projects/mle-mlflow/.venv_flow/lib/python3.10/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to

In [23]:
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, precision_score, recall_score, f1_score, log_loss
# импортируйте необходимые вам модули

# заведите словарь со всеми метриками
metrics = {}

# посчитайте метрики из модуля sklearn.metrics
# err_1 — ошибка первого рода
# err_2 — ошибка второго рода
cm_matrix = confusion_matrix(y_test, prediction, normalize='all').ravel()
err1 = cm_matrix[1]
err2 = cm_matrix[3]
#auc = roc_auc_score(y_test, probas)
#precision = precision_score(y_test, prediction)
#recall = recall_score(y_test, prediction)
#f1 = f1_score(y_test, prediction)
#logloss = log_loss(y_test, prediction) 

# запишите значения метрик в словарь
metrics["err1"] = err1
metrics["err2"] = err2
#metrics["auc"] = auc
#metrics["precision"] = precision
#metrics["recall"] = recall
#metrics["f1"] = f1
#metrics["logloss"] = logloss

In [25]:
loaded_model =  mlflow.sklearn.load_model(model_uri=model_info.model_uri) 


In [4]:
!export S3_BUCKET_NAME=s3-student-mle-20240525-947977af38

In [5]:
import os

import mlflow
import pandas as pd
import joblib


EXPERIMENT_NAME = 'bogomolov'
RUN_NAME = "model_0_registry"
REGISTRY_MODEL_NAME = "churn_model_nikolaibogomolov"

TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000

# напишите код, который подключает tracking и registry uri
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
mlflow.set_registry_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}") 

os.environ["MLFLOW_S3_ENDPOINT_URL"] = 'https://storage.yandexcloud.net'
os.environ["AWS_ACCESS_KEY_ID"] = 'YCAJE8CKXGh8XKUmcmGHpV2CX'
os.environ["AWS_SECRET_ACCESS_KEY"] = 'YCNQmCkW71PeIVypo2YsF-4eIHhiRBZZN94pztrj'
os.environ["S3_BUCKET_NAME"] = 's3-student-mle-20240525-947977af38'

data = pd.read_csv('initial_data.csv')
data.drop(columns = ['building_id', 'studio', 'building_type_int', 'latitude', 'longitude'], inplace = True)
data_test = data[-100:]
X_test = data_test.drop(columns = ['price'])
y_test = data_test['price']

with open('fitted_model.pkl', 'rb') as fd:
    model = joblib.load(fd) 

prediction = y_test
probas = y_test

pip_requirements = 'requirements.txt'
signature = mlflow.models.infer_signature(X_test, prediction)
input_example = X_test[:10]
metadata = {'model_type': 'monthly'}


experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id
if not experiment_id:
    experiment_id = mlflow.create_experiment(EXPERIMENT_NAME).experiment_id

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    run_id = run.info.run_id

    model_info = mlflow.sklearn.log_model( 
        sk_model=model,
        artifact_path="models",
        registered_model_name=REGISTRY_MODEL_NAME,
        signature = signature,
        pip_requirements=pip_requirements,
        input_example = input_example,
        metadata = metadata,
        await_registration_for=60       
    )
    
    

/home/mle-user/mle_projects/mle-mlflow/.venv_flow/lib/python3.10/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator OneHotEncoder from version 1.4.0 when using version 1.3.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/mle-user/mle_projects/mle-mlflow/.venv_flow/lib/python3.10/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.4.0 when using version 1.3.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/mle-user/mle_projects/mle-mlflow/.venv_flow/lib/python3.10/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to

MlflowException: API request to http://127.0.0.1:5000/api/2.0/mlflow/experiments/get-by-name failed with exception HTTPConnectionPool(host='127.0.0.1', port=5000): Max retries exceeded with url: /api/2.0/mlflow/experiments/get-by-name?experiment_name=bogomolov (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f42e2c9b700>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [2]:
import os

import mlflow
import pandas as pd
import joblib


EXPERIMENT_NAME = 'bogomolov'
RUN_NAME = "model_0_registry"
REGISTRY_MODEL_NAME = "churn_model_nikolaibogomolov"

TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000



os.environ["MLFLOW_S3_ENDPOINT_URL"] = 'https://storage.yandexcloud.net'
os.environ["AWS_ACCESS_KEY_ID"] = 'YCAJE8CKXGh8XKUmcmGHpV2CX'
os.environ["AWS_SECRET_ACCESS_KEY"] = 'YCNQmCkW71PeIVypo2YsF-4eIHhiRBZZN94pztrj'
os.environ["S3_BUCKET_NAME"] = 's3-student-mle-20240525-947977af38'

# напишите код, который подключает tracking и registry uri
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
mlflow.set_registry_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}") 

data = pd.read_csv('initial_data.csv')
data.drop(columns = ['building_id', 'studio', 'building_type_int', 'latitude', 'longitude'], inplace = True)
data_test = data[-100:]
X_test = data_test.drop(columns = ['price'])
y_test = data_test['price']

with open('fitted_model.pkl', 'rb') as fd:
    model = joblib.load(fd) 

prediction = y_test
probas = y_test

pip_requirements = 'requirements.txt'
signature = mlflow.models.infer_signature(X_test, prediction)
input_example = X_test[:10]
metadata = {'model_type': 'monthly'}


experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id
if not experiment_id:
    experiment_id = mlflow.create_experiment(EXPERIMENT_NAME).experiment_id

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    run_id = run.info.run_id

    model_info = mlflow.sklearn.log_model( 
        sk_model=model,
        artifact_path="models",
        registered_model_name=REGISTRY_MODEL_NAME,
        signature = signature,
        pip_requirements=pip_requirements,
        input_example = input_example,
        metadata = metadata,
        await_registration_for=60       
    )
    
    

/home/mle-user/mle_projects/mle-mlflow/.venv_flow/lib/python3.10/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator OneHotEncoder from version 1.4.0 when using version 1.3.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/mle-user/mle_projects/mle-mlflow/.venv_flow/lib/python3.10/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.4.0 when using version 1.3.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/mle-user/mle_projects/mle-mlflow/.venv_flow/lib/python3.10/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to

ParamValidationError: Parameter validation failed:
Invalid bucket name "": Bucket name must match the regex "^[a-zA-Z0-9.\-_]{1,255}$" or be an ARN matching the regex "^arn:(aws).*:(s3|s3-object-lambda):[a-z\-0-9]*:[0-9]{12}:accesspoint[/:][a-zA-Z0-9\-.]{1,63}$|^arn:(aws).*:s3-outposts:[a-z\-0-9]+:[0-9]{12}:outpost[/:][a-zA-Z0-9\-]{1,63}[/:]accesspoint[/:][a-zA-Z0-9\-]{1,63}$"